In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


# Word embedding

In [38]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

# 关于word embedding，以序列建模为例
# 考虑source sentence 和 target sentence
# 构建序列，序列的字符以其在词表中的索引的形式表示

# 先把目标序列和源序列的长度假设一个值，然后再根据这个长度再去随机生成单词的索引
batch_size = 2

# 单词表大小
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8 # 原论文中是512

# 序列的最大长度
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5

#src_len = torch.randint(2, 5, (batch_size, ))
#tgt_len = torch.randint(2, 5, (batch_size, ))

src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)
# print(src_len)

# 单词索引(Token ID)构成源句子和目标句子，构建batch，并且做了padding，默认值为0
src_seq = [torch.randint(1, max_num_src_words, (L,)) for L in src_len] # 分别取出src_len中的每一个元素L，然后创建在[1, max_num_src_words-1]之间的(L,)的向量，这一行的结果是一个有两个元素的列表，列表第一个元素为(2, )shape的tensor，第二个元素为(4, )shape的tensor
src_seq_pad = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len -L)), 0) for L in src_len], dim=0) # (0, max_src_seq_len -L)指的是左边不填充，右边填充元素个数为max_src_seq_len -L（默认填充0，但是你可以通过指定 mode 参数'constant', 'reflect', 'replicate', 'circular'来改变填充的元素或者填充的方式）。torch.unsqueeze(input, dim) input：要操作的张量；dim：要增加的维度的索引
tgt_seq = [torch.randint(1, max_num_tgt_words, (L,)) for L in tgt_len]
tgt_seq_pad = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len -L)), 0) for L in tgt_len], dim=0) # dim = 0按行进行拼接

# print("Before padding:", '\n', src_seq)
# print("After padding: ", '\n', src_seq_pad)

# print("-"*100)

# print("Before padding:", '\n', tgt_seq)
# print("After padding:", '\n', tgt_seq_pad)

# print("-"*100)


########################################################## 构造 word embedding ##########################################################
"""为什么要+1：
  在创建嵌入表时，通常需要为每个可能的索引（包括0）提供一个嵌入向量。
  这里的 max_num_src_words 表示源句子中可能出现的最大不同单词的数量。
  如果我们只创建 max_num_src_words 个嵌入向量，那么我们就没有为索引0提供嵌入向量。
  在许多情况下，索引0用于表示特殊的“填充”（padding）标记，这是在处理变长序列时常用的技术。"""
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim) # 实例化一个Embedding表，将离散的输入数据（如单词、类别标签等）映射到连续的向量空间中，第一个参数num_embeddings就是需要嵌入到连续向量空间中的不同项目的数量
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)

src_embedding = src_embedding_table(src_seq_pad) # 将离散的index标签映射到连续的向量空间中
tgt_embedding = tgt_embedding_table(tgt_seq_pad)

# print("The size of src_embedding_table.weight is: ", src_embedding_table.weight)
# print("The result of src_embedding is: ", src_embedding)
# print("The size of src_embedding is: ", src_embedding.shape) # torch.Size([2, 5, 8]), [batch_size, seq_length, embedding_dim]

########################################################## 构造 positional encoding ##########################################################
pos_mat = torch.arange(max_position_len).reshape((-1, 1)) # 行
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1)) / model_dim) # 列
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat / i_mat) # 偶数列，PE(pso, 2i) = sin(pos / 10000^(2*i / d_model))，这里用到了广播机制
pe_embedding_table[:, 1::2] = torch.cos(pos_mat / i_mat) # 奇数列，PE(pos, 2i+1) = cos(pos / 10000^(2*i / d_model))

pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)

# print(pe_embedding.weight)
# print("-"*100)
# print(pe_embedding_table)

src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)), 0) for _ in src_len]).to(torch.int32) # 因为张量src_len中含有两个元素，所以会执行两次操作；缺省参数是dim=0，默认按行拼接
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)), 0) for _ in tgt_len]).to(torch.int32)

# 这里应该传入的是位置索引而不是单词索引，在 Transformer 模型中，单词索引首先被传递给单词嵌入层，得到每个单词的嵌入向量。而由于模型本身是无序的（即不考虑输入序列的顺序），位置索引用于提供这种顺序信息
src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)

# print(src_pos)
# print("-"*100)
# print(src_pe_embedding)

# print(tgt_pos)
# print("-"*100)
# print(tgt_pe_embedding)

########################################################## softmax演示：scaled的重要性 ##########################################################
alpha1 = 0.1
alpha2 = 10
score = torch.randn(5)
# print(score)
prob1 = F.softmax(score * alpha1, -1) # -1 参数表示 softmax 操作将沿着输入张量的最后一个维度进行。例如，假设 score 是一个形状为 [batch_size, num_classes] 的张量，其中 num_classes 是类别的数量。调用 F.softmax(score, -1) 将会对每个样本（即每一行）的 num_classes 分数进行 softmax 转换，得到每个样本对应每个类别的概率分布。
prob2 = F.softmax(score * alpha2, -1)
# print("prob1 = ", prob1) # 可以看出prob1的值之间的差距并不太大
# print("prob2 = ", prob2) # prob2之间的差距就非常大，达到数量级
"""
上面prob1, prob2主要是为了演示在Transformer中为什么会除以一个因子 sqrt(dk)，除以 sqrt(dk) 可以使得 QKT 的结果的方差接近于1，这有助于保持 softmax 函数输出的稳定性和一致性
如果没有这种归一化，那么在高维空间中，点积的值可能会过大，因为点积的值是所有维度上对应元素乘积的和，这会导致 softmax 函数的输出偏向于某些特定的元素，从而失去注意力机制的多样性
"""

def softmax_func(score):
    return F.softmax(score)
jaco_mat1 = torch.autograd.functional.jacobian(softmax_func, score*alpha1) # 相当于反向传播过程中，链式求导中间的一个过程
jaco_mat2 = torch.autograd.functional.jacobian(softmax_func, score*alpha2)

# print("jaco_mat1 = ", jaco_mat1) # 可以看出梯度信息还是相对平稳的
# print("-"*100)
# print("jaco_mat2 = ", jaco_mat2) # 可以看出部分梯度都很接近于0，训练过程中梯度的更新就会比较缓慢

"""
对于Transformer中计算的Attention(Q, K, V) = softmax((Q * KT) / sqrt(dk)) * V, 数学上可以证明 Q * KT 的方差是 dk，这个方差是比较大的，所以可以除以sqrt(dk)使其方差变为1
"""



########################################################## 构造encoder的self-attention mask ##########################################################
# mask的shape：[batch_size, max_src_len, max_src_len] (输入的n个token数量相同的n*n矩阵，代表token之间的关联性，因为encoder算的是self- attention), 并且它的值为 1 或 -inf




jaco_mat1 =  tensor([[ 0.1330, -0.0332, -0.0349, -0.0347, -0.0302],
        [-0.0332,  0.1660, -0.0464, -0.0462, -0.0402],
        [-0.0349, -0.0464,  0.1720, -0.0485, -0.0422],
        [-0.0347, -0.0462, -0.0485,  0.1715, -0.0421],
        [-0.0302, -0.0402, -0.0422, -0.0421,  0.1547]])
----------------------------------------------------------------------------------------------------
jaco_mat2 =  tensor([[ 1.6778e-15, -7.3970e-18, -1.0150e-15, -6.5537e-16, -6.1968e-22],
        [-7.3970e-18,  4.3893e-03, -2.6672e-03, -1.7221e-03, -1.6283e-09],
        [-1.0150e-15, -2.6672e-03,  2.3898e-01, -2.3631e-01, -2.2344e-07],
        [-6.5537e-16, -1.7221e-03, -2.3631e-01,  2.3803e-01, -1.4427e-07],
        [-6.1968e-22, -1.6283e-09, -2.2344e-07, -1.4427e-07,  3.6934e-07]])


/var/folders/qh/9_9y_gnj0nz554cp2nc9_96w0000gn/T/ipykernel_68323/2147454232.py:107: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(score)


# torch.nn.Embedding()

In [ ]:
# torch.nn.Embedding()

import torch
import torch.nn as nn

embedding = nn.Embedding(10, 3) # 定义一个embedding模块，包含了一个长度为10的张量，每个张量的大小是3
# print(embedding)
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
e = embedding(input)
print(e) # e.shape: [2, 4, 3], 在经过nn.embedding后，从[2, 4]维度变换为[2, 4, 3]，其实就是[2, 4]中的每个值作为索引去nn.embedding中取对应的权重
print("The shape of e is: ", e.shape)
print(e[1][3]) # [0.3204,  1.6376, -0.4442]，相应的3即为embedding后的权重
print("-"*100)
print(embedding.weight) 
print("The shape of embedding.weight is: ", embedding.weight.shape)

tensor([[[-1.4551,  0.3129,  1.1398],
         [-2.1301,  0.4180,  0.4204],
         [-0.1716, -1.8632,  0.3244],
         [-0.5084, -1.6552, -1.8416]],

        [[-0.1716, -1.8632,  0.3244],
         [ 0.6709,  0.4043, -2.2828],
         [-2.1301,  0.4180,  0.4204],
         [ 0.3204,  1.6376, -0.4442]]], grad_fn=<EmbeddingBackward0>)
The shape of e is:  torch.Size([2, 4, 3])
tensor([ 0.3204,  1.6376, -0.4442], grad_fn=<SelectBackward0>)
----------------------------------------------------------------------------------------------------
Parameter containing:
tensor([[-1.7047,  0.8644, -0.4895],
        [-1.4551,  0.3129,  1.1398],
        [-2.1301,  0.4180,  0.4204],
        [ 0.6709,  0.4043, -2.2828],
        [-0.1716, -1.8632,  0.3244],
        [-0.5084, -1.6552, -1.8416],
        [-1.5301,  0.2444, -0.6057],
        [ 2.0089,  1.4314,  0.9333],
        [-2.0367,  0.4912,  0.7898],
        [ 0.3204,  1.6376, -0.4442]], requires_grad=True)
The shape of embedding.weight is:  torch.Si